In [ ]:
%matplotlib widget

#Python libraries
import os                            # find files
from matplotlib import pyplot as plt # plotting
import numpy as np                   # math library
from tqdm import tqdm           # progress bar
from collections import defaultdict  # special dictionary type
from time import time                # time module

# Ping imports
import themachinethatgoesping as ping                       # main module
import themachinethatgoesping.echosounders.simrad as simrad # simrad submodule
import themachinethatgoesping.tools as tools # simrad submodule

print(ping.version())

In [ ]:
#load files
folders=[]
with open('test_data.txt','r') as ifi:
    for line in ifi:
        print("Folder:", line)
        folders.append(line)

if not len(folders):
    print("Please add test_data folders to 'test_data.txt'")

#find .raw files in "folders"
files = []
for folder in folders:
    for r,d,f in os.walk(folder):
        for file in f:
            if file.endswith('.raw'):
                files.append(r + '/' +file)
                
files.sort()
#for f in files:
#    print(f)
    
#print()


In [ ]:
def progressbar(ncols = 140):
    return tools.progressbars.ProgressTqdm(tqdm(ncols=ncols))

In [ ]:
# Open raw files
#fm = simrad.FileSimradRaw_mapped(files, False, progressbar())
fm = simrad.FileSimradRaw_mapped(files, init = False)

fm.navigation_interface.set_min_gga_quality(4)

fm.init_interfaces()

print(fm)

In [ ]:
fm.datagram_interface.datagram_headers().print()

In [ ]:
ping = fm.pings()[100]

In [ ]:
ping.print()

In [ ]:
ping.raw_data.print()

In [ ]:

containers = fm.pings().get_sorted_by_time().break_by_time_diff(3600)

for i,container_pings in enumerate(containers):
    print(f"-- container {i} --")
    
    for cid in container_pings.find_channel_ids():
        print(f" - {cid}: {len(container_pings(cid))}")
    print()

In [ ]:
fpings = containers[-2](fm.channel_ids()[-1])
#fpings = fm.pings().get_sorted_by_time()(channel_ids[-1])
print(fpings)

In [ ]:
fm.datagram_interface.datagrams("XML0")[11].print()

In [ ]:
ping = fpings[1000]

In [ ]:
ping.get_file_path()

In [ ]:
pings = fpings[::1]
pings = fpings[::1]

nsamples = min(3000,pings.max_number_of_samples())

image = np.empty((len(pings),nsamples),dtype=np.float32)
image.fill(np.nan)

for i,ping in enumerate(tqdm(pings, ncols = 120)):
    sv = ping.get_sv_stacked(dB = True)
    ns = min(nsamples,len(sv))
    image[i,:ns] = sv[:ns]
    
plt.close("echogram")
fig = plt.figure("echogram",figsize=(12,4))
ax = fig.subplots()
mapped = ax.imshow(image.transpose(),aspect='auto',cmap='YlGnBu_r', vmin = -100, vmax=-30)
fig.colorbar(mapped)
fig.tight_layout()

In [ ]:
print(pings)

In [ ]:
plt.close("time")
plt.figure("time")
times = []
for p in fpings:
    times.append(p.get_timestamp())
    
plt.plot(times,marker='+')

In [ ]:
tools.timeconv.unixtime_to_datestring(fpings[0].get_timestamp())

In [ ]:
tools.timeconv.unixtime_to_datestring(fpings[-1].get_timestamp())

In [ ]:
plt.close("test")
plt.figure("test")

times = []
for ping in fpings:
    times.append(ping.get_timestamp())
    
plt.plot(times)
print((max(times)-min(times))/3600)